<center>软件03班 陈启乾 2020012385<center/>

# 实验一：让吃豆人吃到一个食物

In [ ]:
# 小迷宫 + dfs
%run pacman.py -l tinyMaze -p SearchAgent -a fn=depthFirstSearch

In [ ]:
# 中等迷宫 + BFS
%run pacman.py -l mediumMaze -p SearchAgent -a fn=breadthFirstSearch

In [ ]:
# 中等迷宫 + UCS
%run pacman.py -l mediumMaze -p SearchAgent -a fn=uniformCostSearch

In [ ]:
# 大迷宫 + A*
%run pacman.py -l bigMaze -z .5 -p SearchAgent -a fn=aStarSearch,heuristic=manhattanHeuristic

## 实现解读

### DFS

`dfs` 实现在 [search.py](search.py) 的 `depthFirstSearch` 中。

`dfs `使用递归的实现，在递归变量中记录当前 `pos` ，返回值中记录 `action` 的序列。 

### BFS

`bfs` 实现在 [search.py](search.py) 的 `breadthFirstSearch` 中。

`bfs` 采用一个 FIFO 队列实现，每次从队列头中取出元素，然后将扩展的元素加入队列尾，`action` 的序列在“状态”中维护。

### UCS

`ucs` 实现在 [search.py](search.py) 的 `uniformCostSearch` 中。

`ucs` 采用一个优先队列实现，每次从队列头取出元素，然后将还未从队列中取出扩展的元素加入队列，代价函数是给出的，这里是恒为 1 的函数。

### A*

`A*` 搜索实现在 [search.py](search.py) 的 `aStarSearch` 中。

A* 的启发函数，直接使用了曼哈顿距离。即为 $d = |x_1-x_2| + |y_1 - y_2|$ 。

容易发现这个启发函数是良定义的。

1. 满足启发值小于真实值：至少需要走 $d$ 步才可能抵达终点。
2. 三角形不等式：曼哈顿距离显然拥有三角形不等式。



## 表现评估

In [ ]:
%run pacman.py -l maze_gen_500 -p SearchAgent -a fn=breadthFirstSearch -q

In [ ]:
%run pacman.py -l maze_gen_500 -p SearchAgent -a fn=depthFirstSearch -q

In [ ]:
%run pacman.py -l maze_gen_500 -p SearchAgent -a fn=aStarSearch,heuristic=manhattanHeuristic -q

In [ ]:
%run pacman.py -l maze_gen_500 -p SearchAgent -a fn=uniformCostSearch -q

# 实验二：迷宫中存在多个食物，甚至怪物，找到一条尽可能获得高分的路径。

In [ ]:
# 有怪物的，只有一个食物
%run pacman.py -l mediumScaryMaze -p MySearchAgent -a fn=aStarSearch,heuristic=manhattanHeuristic,prob=MediumScarySearchProblem

In [ ]:
# 有很多食物的，没有怪物
%run pacman.py -l foodSearchMaze -p MySearchAgent

## 设计思路

其实这里并非应该换 `agent`，而是应该换 `problem`，因为这里已经不是一个 `position search problem`（路径搜索问题），而分别是躲怪物和吃食物的两个问题。

我们在 [searchAgents.py](searchAgents.py) 中继承了 `PositionSearchProblem`，分别派生了 `FoodSearchProblem` 和 `MediumScarySearchProblem`，完成了这两个问题。

我们在不同的问题中改变了**目标的位置坐标**和**位置的代价函数**，来让吃豆人走出我们需要的路线。

### 第一个问题：MediumScary Maze

这个问题要求我们避开所有的怪物，到达右下角的终点。我们经过观察发现，怪物集中在右下角，因此我们就把右下侧的点的 cost 设置为 1000，其他店设置为 1。

这样我们就可以绕过怪物，从左上侧吃到食物

### 第二个问题：FoodSearch

这个问题要求我们吃到所有的食物。我们注意到：食物分布在上、下、左三侧边上。因此，我们将中间部分和右边部分的 cost 设置为 1000，其他位置设置为 1 。同时，我们将目标设置在右下角。

这样我们就可以依次经过：右上-左上-左下-右下角，吃到所有怪物。


# 实验三：地图中存在一些聪明的怪物的情况，吃豆人的目标是获取尽量高分

In [ ]:
# mini-max 搜索
%run pacman.py -p MinimaxAgent -l mediumClassic.lay -a depth=3,evalFn="myScoreEvaluationFunction"

In [ ]:
# alpha-beta 剪枝
%run pacman.py -p AlphaBetaAgent -l mediumClassic.lay -a depth=5,evalFn="myScoreEvaluationFunction"

## 实现解读

这里分别实现了 MiniMax 对抗搜索和 Alpha-Beta 的剪枝算法。

在 [multiAgents.py](multiAgents.py) 文件中，我们在 `MinimaxAgent` 类和 `AlphaBetaAgent` 中，分别实现了两种算法，重写了 `getAction` 接口。


## 表现评估




## 摆烂问题的解决

值得提到的是，如果直接使用提供的 `scoreEvaluationFunction` 函数，则很容易因为没有怪物在旁边，搜索函数觉得往哪里走都差不多，很多步之后总能吃到附近的豆子，所以在原地不断打转的问题。为了解决这种问题，我们需要让 pacman 对整体局面有一个了解。因此，我们重写了 `myScoreEvaluationFunction`，给每个食物都赋予了一个权值，近处的很高而远处的较低，这样我们的吃豆人就可以更好完成任务。

所以我们得到这样的人生经验，某一个特定方向诱惑越大、欲望越多，就越不容易摆烂。